In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

df = pd.read_excel('Dados/Dados_Oiv/Dados_Oiv.xlsx', 
                   engine='openpyxl')
df.head()

In [ ]:


# Filtrar dados para a região global e remover valores nulos
df_global = df[df["Region/Country"] == "Global"].dropna(subset=["Quantity"])

df_global.head()

In [ ]:

# Filtrar dados para a região global e remover valores nulos
df_global = df[df["Region/Country"] == "Global"].dropna(subset=["Quantity"])

# Converter quantidade de 1000 hl para litros (1 hl = 100 L → 1000 hl = 100,000 L)
df_global["Quantity_Liters"] = df_global["Quantity"] * 100000

# Filtrar os últimos 15 anos (assumindo que o último ano é 2024)
anos_validos = list(range(2010, 2025))
df_global = df_global[df_global["Year"].isin(anos_validos)]

# Função para calcular as estatísticas formatadas
def calcular_estatisticas_formatado(df, variavel):
    df_filtrado = df[df["Variable"] == variavel]
    volume_total = df_filtrado["Quantity_Liters"].sum() / 1e9  # bilhões de litros
    media_anual = df_filtrado["Quantity_Liters"].mean() / 1e6  # milhões de litros
    volume_ultimo_ano = df_filtrado[df_filtrado["Year"] == df_filtrado["Year"].max()]["Quantity_Liters"].sum() / 1e6
    volume_ano_anterior = df_filtrado[df_filtrado["Year"] == df_filtrado["Year"].max() - 1]["Quantity_Liters"].sum() / 1e6
    crescimento = ((volume_ultimo_ano - volume_ano_anterior) / volume_ano_anterior * 100) if volume_ano_anterior > 0 else None
    return {
        "Volume Total (Bilhões L)": round(volume_total, 2),
        "Média Anual (Milhões L)": round(media_anual, 2),
        "Volume Último Ano (Milhões L)": round(volume_ultimo_ano, 2),
        "Crescimento Último Ano (%)": round(crescimento, 2) if crescimento is not None else None
    }

# Calcular estatísticas para cada alavanca
estatisticas_producao = calcular_estatisticas_formatado(df_global, "Production")
estatisticas_importacao = calcular_estatisticas_formatado(df_global, "Imports")
estatisticas_exportacao = calcular_estatisticas_formatado(df_global, "Exports")
estatisticas_consumo = calcular_estatisticas_formatado(df_global, "Consumption")

# Criar DataFrame consolidado
tabela_resultado_formatada = pd.DataFrame({
    "Produção": estatisticas_producao,
    "Importação": estatisticas_importacao,
    "Exportação": estatisticas_exportacao,
    "Consumo": estatisticas_consumo
}).T

In [ ]:
tabela_resultado_formatada.head()

In [ ]:
# Ajustar a tabela para melhor visualização (como print)
tabela_formatada_print = tabela_resultado_formatada.copy()
tabela_formatada_print.reset_index(inplace=True)
tabela_formatada_print.rename(columns={"index": "Alavanca"}, inplace=True)

# Formatando os valores para exibição (ex: 10.000,00 com vírgula e %)
tabela_formatada_print["Volume Total (Bilhões L)"] = tabela_formatada_print["Volume Total (Bilhões L)"].map("{:,.2f}".format)
tabela_formatada_print["Média Anual (Milhões L)"] = tabela_formatada_print["Média Anual (Milhões L)"].map("{:,.2f}".format)
tabela_formatada_print["Volume Último Ano (Milhões L)"] = tabela_formatada_print["Volume Último Ano (Milhões L)"].map("{:,.2f}".format)
tabela_formatada_print["Crescimento Último Ano (%)"] = tabela_formatada_print["Crescimento Último Ano (%)"].map("{:.2f}%".format)

display(tabela_formatada_print)
